In [27]:
import folium
import pandas as pd
import numpy as np
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [5]:
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
df = pd.read_csv(url)
df.head()

,Flight Number,Date,Time (UTC),Booster Version,Launch Site,Payload,Payload Mass (kg),Orbit,Customer,Landing Outcome,class,Lat,Long
0,1,2010-06-04,18:45:00,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0.0,LEO,SpaceX,Failure (parachute),0,28.562302,-80.577356
1,2,2010-12-08,15:43:00,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel o...",0.0,LEO (ISS),NASA (COTS) NRO,Failure (parachute),0,28.562302,-80.577356
2,3,2012-05-22,7:44:00,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2+,525.0,LEO (ISS),NASA (COTS),No attempt,0,28.562302,-80.577356
3,4,2012-10-08,0:35:00,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356
4,5,2013-03-01,15:10:00,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677.0,LEO (ISS),NASA (CRS),No attempt,0,28.562302,-80.577356


In [7]:
spacex_df = df[['Launch Site', 'Lat', 'Long', 'class']]
launchsite_df = spacex_df.groupby(['Launch Site'],as_index=False).first()
launchsite_df = launchsite_df[['Launch Site', 'Lat', 'Long']]
launchsite_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [11]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate,zoom_start=10)

In [21]:
circle = folium.Circle(nasa_coordinate,radius=1000,color='#d35400',fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
marker = folium.map.Marker(
    nasa_coordinate,icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC'))
site_map.add_child(circle)
site_map.add_child(marker)

In [17]:
unique_launch_sites = launchsite_df['Launch Site'].unique()
unique_launch_sites

array(['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'],
      dtype=object)

## TASK 1:
### Mark all launch sites on a map.

In [19]:
launch_sites_data = {
    'Launch Site': [
        'CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'
    ],
    'Lat': [
        28.562302, 28.563197, 28.573255, 34.632834
    ],
    'Long': [
        -80.577356, -80.576820, -80.646895, -120.610745
    ]
}

# Create the DataFrame
launch_sites_df = pd.DataFrame(launch_sites_data)

# Create a map centered around the average coordinates of the launch sites
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Mark all launch sites on the map
for _, row in launch_sites_df.iterrows():
    folium.CircleMarker(
        location=(row['Lat'], row['Long']),
        radius=8,  # Adjust the radius as needed
        color='blue',  # Circle color
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=row['Launch Site']  # Popup label with the launch site name
    ).add_to(site_map)

# Display the map
site_map.save('launch_sites_map.html')  # Save the map to an HTML file
site_map

## TASK 2:
### Mark the success/failed launches for each site on the map.

In [36]:
# Sample launch sites data
launch_sites_data = {
    'Launch Site': [
        'CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'
    ],
    'Lat': [
        28.562302, 28.563197, 28.573255, 34.632834
    ],
    'Long': [
        -80.577356, -80.576820, -80.646895, -120.610745
    ],
    'Success': [
        True, False, True, False
    ]
}

# Create DataFrame
launch_sites_df = pd.DataFrame(launch_sites_data)

# Create a map centered around the average coordinates of the launch sites
nasa_coordinate = [launch_sites_df['Lat'].mean(), launch_sites_df['Long'].mean()]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# Mark all launch sites on the map with success/failure
for _, row in launch_sites_df.iterrows():
    color = 'green' if row['Success'] else 'red'  # Green for success, red for failure
    
    folium.Marker(
        location=(row['Lat'], row['Long']),
        popup=f"{row['Launch Site']} - {'Success' if row['Success'] else 'Failed'}",
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)  # Add marker to the cluster

# Display the map
site_map.save('launch_sites_clustered_map.html')  # Save the map to an HTML file
site_map

## TASK 3:
### Calculate the distances between a launch site to its proximities.

In [40]:
launch_sites_data = {
    'Launch Site': [
        'CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E'
    ],
    'Lat': [
        28.562302, 28.563197, 28.573255, 34.632834
    ],
    'Long': [
        -80.577356, -80.576820, -80.646895, -120.610745
    ]
}

# Create DataFrame
launch_sites_df = pd.DataFrame(launch_sites_data)

# Haversine function to calculate the distance between two points
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Radius of the Earth in km
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = (np.sin(dlat / 2) ** 2 +
         np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Create an empty DataFrame for distances
distances = pd.DataFrame(columns=['From', 'To', 'Distance (km)'])

# Calculate distances between each launch site and others
for i in range(len(launch_sites_df)):
    for j in range(len(launch_sites_df)):
        if i != j:  # Avoid calculating distance to itself
            lat1 = launch_sites_df.iloc[i]['Lat']
            lon1 = launch_sites_df.iloc[i]['Long']
            lat2 = launch_sites_df.iloc[j]['Lat']
            lon2 = launch_sites_df.iloc[j]['Long']
            distance = haversine(lat1, lon1, lat2, lon2)
            
            # Create a DataFrame for the new distance entry
            new_distance = pd.DataFrame([{
                'From': launch_sites_df.iloc[i]['Launch Site'],
                'To': launch_sites_df.iloc[j]['Launch Site'],
                'Distance (km)': distance
            }])
            
            # Concatenate the new distance entry to the distances DataFrame
            distances = pd.concat([distances, new_distance], ignore_index=True)

# Display the distances DataFrame
print(distances)

            From            To  Distance (km)
0    CCAFS LC-40  CCAFS SLC-40       0.112447
1    CCAFS LC-40    KSC LC-39A       6.899330
2    CCAFS LC-40   VAFB SLC-4E    3825.840276
3   CCAFS SLC-40   CCAFS LC-40       0.112447
4   CCAFS SLC-40    KSC LC-39A       6.934085
5   CCAFS SLC-40   VAFB SLC-4E    3825.854446
6     KSC LC-39A   CCAFS LC-40       6.899330
7     KSC LC-39A  CCAFS SLC-40       6.934085
8     KSC LC-39A   VAFB SLC-4E    3819.052759
9    VAFB SLC-4E   CCAFS LC-40    3825.840276
10   VAFB SLC-4E  CCAFS SLC-40    3825.854446
11   VAFB SLC-4E    KSC LC-39A    3819.052759


C:\Users\ELITRBOOK\AppData\Local\Temp\ipykernel_7736\1311918823.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  distances = pd.concat([distances, new_distance], ignore_index=True)
